# Environment

>pip install notebook pandas selenium beautifulsoup4

In [1]:
import pandas                                     # for manipulating dataframes
from datetime import datetime                     # for manipulating dates

# for web-scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

In [2]:
NUM_PAGES = 13
assert NUM_PAGES > 1, "NUM_PAGES should be more than 1."

from selenium import webdriver
from selenium.webdriver.common.by import By

links = []

# opens Chrome window
driver = webdriver.Chrome()

# get the links on the first page
driver.get("https://pubmed.ncbi.nlm.nih.gov/?term=%28%28%22machine+learning%22%5BMesh%5D+OR+%22supervised+machine+learning%22%5BMesh%5D+OR+%22deep+learning%22%5BMesh%5D+OR+%22unsupervised+machine+learning%22%5BMesh%5D+OR+artificial+intelligence%5Btiab%5D+OR+Twitter%5Btiab%5D+OR+social+media%5Btiab%5D%29+AND+%28%22intimate+partner+violence%22%5BMesh%5D+OR+%22gender-based+violence%22%5BMesh%5D+OR+%22domestic+violence%22%5BMesh%5D+OR+intimate+partner+violence%5Btiab%5D+OR+domestic+violence%5Btiab%5D+OR+child+abuse%5Btiab%5D+OR+spouse+abuse%5Btiab%5D%29%29&filter=simsearch2.ffrft&filter=datesearch.y_10")
post_links = driver.find_elements(By.CSS_SELECTOR, 'article a')
for link in post_links:
    links.append(link.get_attribute("href"))

# get the links on subsequent pages
for page_num in range(2, NUM_PAGES + 1):
    driver.get("https://pubmed.ncbi.nlm.nih.gov/?term=((%22machine%20learning%22%5BMesh%5D%20OR%20%22supervised%20machine%20learning%22%5BMesh%5D%20OR%20%22deep%20learning%22%5BMesh%5D%20OR%20%22unsupervised%20machine%20learning%22%5BMesh%5D%20OR%20artificial%20intelligence%5Btiab%5D%20OR%20Twitter%5Btiab%5D%20OR%20social%20media%5Btiab%5D)%20AND%20(%22intimate%20partner%20violence%22%5BMesh%5D%20OR%20%22gender-based%20violence%22%5BMesh%5D%20OR%20%22domestic%20violence%22%5BMesh%5D%20OR%20intimate%20partner%20violence%5Btiab%5D%20OR%20domestic%20violence%5Btiab%5D%20OR%20child%20abuse%5Btiab%5D%20OR%20spouse%20abuse%5Btiab%5D))&filter=simsearch2.ffrft&filter=datesearch.y_10&page=" + str(page_num))
    post_links = driver.find_elements(By.CSS_SELECTOR, 'article div div a')
    for link in post_links:
        links.append(link.get_attribute("href"))

# closes Chrome window
driver.quit()

In [3]:
def save_list_to_file(lst, filename):
    """
    Takes a list of links and turns each one to a file.
    """
    with open(filename, 'w', encoding='utf-8') as f:
        for item in lst:
            f.write(str(item) + '\n')

In [4]:
def current_time_to_serial():
    """
    Returns the current timestamp in serial format.
    """
    current_time = datetime.now()
    return current_time.timestamp()

In [5]:
for post_link in links:
    soup = BeautifulSoup(requests.get(post_link).content, 'html.parser')
   
    # find the first element of the specified class
    content = soup.find('div', class_='abstract')

    texts = []
    
    # extracts all the texts from the paragraphs found within the class
    paragraphs = content.find_all('p')
    paragraph_texts = [p.get_text().replace('\n', '') for p in paragraphs]
    texts.append(paragraph_texts)
    
    # saves each abstract as a file
    save_list_to_file(texts, "../data/abstract/domestic-violence-abstract-" + str(current_time_to_serial()) + ".txt")